In [ ]:
import sagemaker
from sagemaker.sklearn.estimator import SKLearn
from sagemaker.tuner import IntegerParameter, ContinuousParameter, CategoricalParameter, HyperparameterTuner
from sagemaker import get_execution_role
import os

print("--- ⚙️ Step 1: Initializing SageMaker Environment ---")

# 1. 获取执行角色 (IAM Role)
# 如果在本地 VS Code 运行，可能需要手动指定 Role ARN
try:
    role = get_execution_role()
except ValueError:
    print("⚠️ 警告: 无法自动获取角色，请手动填入 ARN。")
    # 请去 AWS Console -> IAM -> Roles 找一个类似 AmazonSageMaker-ExecutionRole 的角色
    role = "arn:aws:iam::137568342316:role/SageMakerExecutionRole" 

# 2. 基础配置
sagemaker_session = sagemaker.Session()
bucket_name = 'sleep-disorder-mlops-bucket' # 您的 S3 桶名

# 3. 指定代码源 (Source of Truth for Code)
# SageMaker 会自动 `git clone` 这个仓库到训练实例中
git_repo = 'https://github.com/vinnie071015/sleeping-disorder-mlops.git' # 替换为您的仓库地址
git_config = {
    'repo': git_repo, 
    'branch': 'main'
}

print(f"✅ Role: {role}")
print(f"✅ Bucket: {bucket_name}")
print(f"✅ Git Repo: {git_repo}")


print("\n--- 💾 Step 2: Defining S3 Data Input ---")

# 定义 S3 数据输入
# SageMaker 会自动把这个 S3 路径下的所有文件下载到容器内的 /opt/ml/input/data/train/
# 这里的 s3_data 必须指向包含 sleep_data.csv 的文件夹路径 (以 / 结尾)
s3_input_path = f's3://{bucket_name}/raw_data/'

train_input = sagemaker.inputs.TrainingInput(
    s3_data=s3_input_path, 
    content_type='text/csv'
)

print(f"✅ Training Data Source: {s3_input_path}")


print("\n--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---")

# 使用 ml.m5.large (通用型)
instance_type = 'ml.m5.large'

# ⚠️ 关键修复：定义 S3 模型输出路径，强制使用你的自定义桶
model_output_s3_path = f's3://{bucket_name}/sagemaker-tuning-output/' 
print(f"✅ Model Artifacts Output Path (FIXED): {model_output_s3_path}")


common_estimator_args = {
    # 修正 1: 入口文件路径要包含 src (因为 source_dir 变成了根目录)
    'entry_point': 'src/train.py',
    
    # 修正 2: source_dir 设为 '.' (代表 Git 仓库的根目录)
    'source_dir': '.',
    
    'role': role,
    'instance_count': 1,
    'instance_type': instance_type,
    'framework_version': '1.2-1',
    'py_version': 'py3',
    'git_config': git_config,
    'sagemaker_session': sagemaker_session,
    
    # === 关键修复：添加 output_path 参数 ===
    'output_path': model_output_s3_path,
    'environment': {
        'WANDB_API_KEY': "0f759a15e3c54016f3f727c9720f0c9206fdd5c1",  # 容器会自动读取这个变量进行登录
        'WANDB_PROJECT': 'sleep-disorder-mlops', # 可选：指定项目名
        'WANDB_WATCH': 'false' # 可选：关闭不必要的模型监控以加快速度
    }
    # ========================================
}

print(f"✅ Instance Type: {instance_type}")
print(f"✅ Path Correction: source_dir='.', entry_point='src/train.py'")


print("\n--- 🎛️ Step 4: Defining Tuners for 3 Models ---")

# 定义指标抓取规则 (对应 src/train.py 中的 print 语句)
metric_definitions = [
    {'Name': 'accuracy', 'Regex': '✅ Accuracy: ([0-9\\.]+)'},
    {'Name': 'f1', 'Regex': '✅ F1 Score: ([0-9\\.]+)'}
]

# ==========================================
# A. Random Forest Tuner
# ==========================================
# 注意：这里使用的 common_estimator_args 已经包含了 output_path
rf_estimator = SKLearn(**common_estimator_args) 
# 固定模型类型为 RF
rf_estimator.set_hyperparameters(model_type='random_forest')

rf_tuner = HyperparameterTuner(
    estimator=rf_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'n_estimators': IntegerParameter(50, 150),
        'max_depth': IntegerParameter(5, 15)
    },
    max_jobs=2,          # 总共跑 2 次 (省钱)
    max_parallel_jobs=1, # 串行跑 (安全)
    base_tuning_job_name='rf-tuning'
)

# ==========================================
# B. SVM Tuner
# ==========================================
svm_estimator = SKLearn(**common_estimator_args)
svm_estimator.set_hyperparameters(model_type='svm')

svm_tuner = HyperparameterTuner(
    estimator=svm_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0),
        'kernel': CategoricalParameter(['rbf', 'linear'])
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='svm-tuning'
)

# ==========================================
# C. Logistic Regression Tuner
# ==========================================
lr_estimator = SKLearn(**common_estimator_args)
lr_estimator.set_hyperparameters(model_type='logistic_regression')

lr_tuner = HyperparameterTuner(
    estimator=lr_estimator,
    objective_metric_name='accuracy',
    metric_definitions=metric_definitions,
    hyperparameter_ranges={
        'C': ContinuousParameter(0.1, 5.0)
    },
    max_jobs=2,
    max_parallel_jobs=1,
    base_tuning_job_name='lr-tuning'
)

print("✅ Tuners for RF, SVM, and LR are ready.")

sagemaker.config INFO - Not applying SDK defaults from location: /Library/Application Support/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /Users/zhengchengsheng/Library/Application Support/sagemaker/config.yaml
--- ⚙️ Step 1: Initializing SageMaker Environment ---


Couldn't call 'get_role' to get Role ARN from role name ML_Project to get Role path.


⚠️ 警告: 无法自动获取角色，请手动填入 ARN。
✅ Role: arn:aws:iam::137568342316:role/SageMakerExecutionRole
✅ Bucket: sleep-disorder-mlops-bucket
✅ Git Repo: https://github.com/vinnie071015/sleeping-disorder-mlops.git

--- 💾 Step 2: Defining S3 Data Input ---
✅ Training Data Source: s3://sleep-disorder-mlops-bucket/raw_data/

--- 🏗️ Step 3: Defining Common Estimator Configuration (Git Mode) ---
✅ Model Artifacts Output Path (FIXED): s3://sleep-disorder-mlops-bucket/sagemaker-tuning-output/


NameError: name 'wandb_api_key' is not defined

In [ ]:
import boto3
import time
from datetime import datetime
import sagemaker

# ==========================================
# 🛠️ 辅助函数定义
# ==========================================

def get_tuning_job_status(tuner_obj):
    """获取 Tuner 当前 Job 的状态"""
    try:
        job_name = tuner_obj.latest_tuning_job.name
        response = boto3.client('sagemaker').describe_hyper_parameter_tuning_job(HyperParameterTuningJobName=job_name)
        return response['HyperParameterTuningJobStatus']
    except Exception:
        return "Initializing"

def fetch_latest_error_log(bucket_name, prefix="debug_logs/"):
    """
    [失败时调用] 从 S3 获取最新生成的调试日志并打印
    """
    print(f"\n🔍 [自动诊断] 检测到失败，正在从 s3://{bucket_name}/{prefix} 拉取最新日志...")
    s3 = boto3.client('s3')
    try:
        response = s3.list_objects_v2(Bucket=bucket_name, Prefix=prefix)
        if 'Contents' not in response:
            print("⚠️ 未在 S3 找到日志文件。")
            return
        
        # 按最后修改时间排序，取最新的
        files = sorted(response['Contents'], key=lambda x: x['LastModified'])
        latest_file = files[-1]
        key = latest_file['Key']
        
        print(f"📄 发现最新日志: {key} (时间: {latest_file['LastModified']})")
        print("="*60)
        file_obj = s3.get_object(Bucket=bucket_name, Key=key)
        print(file_obj['Body'].read().decode('utf-8')) # 打印日志内容
        print("="*60)
    except Exception as e:
        print(f"❌ 拉取日志失败: {e}")

def print_best_model_info(tuner_obj, model_name):
    """
    [成功时调用] 获取并打印最佳模型的 S3 路径
    """
    try:
        best_job = tuner_obj.best_training_job()
        if not best_job:
            print(f"⚠️ {model_name}: 任务显示完成，但未找到最佳 Training Job (可能是所有子任务都失败了)。")
            return

        # 获取详细信息
        desc = best_job.describe()
        score = desc['FinalMetricDataList'][0]['Value']
        s3_uri = desc['ModelArtifacts']['S3ModelArtifacts']
        
        print(f"\n🏆 {model_name} 训练完成！")
        print(f"   ✅ 最佳准确率 (Accuracy): {score:.4f}")
        print(f"   💾 最佳模型保存位置: {s3_uri}")
    except Exception as e:
        print(f"⚠️ 获取 {model_name} 模型信息失败: {e}")

# ==========================================
# 🚀 主流程：并行启动
# ==========================================

print("\n--- 🚀 Step 5: Launching All Tuning Jobs in Parallel (Smart Monitor) ---")
print(f"🕒 开始时间: {datetime.now().strftime('%H:%M:%S')}\n")

# 1. 启动 Random Forest
print(f"🌲 [1/3] Launching Random Forest...", end=" ")
rf_tuner.fit({'train': train_input}, wait=False) 
print(f"✅ Job ID: {rf_tuner.latest_tuning_job.name}")

# 2. 启动 SVM
print(f"🛡️ [2/3] Launching SVM...", end=" ")
svm_tuner.fit({'train': train_input}, wait=False)
print(f"✅ Job ID: {svm_tuner.latest_tuning_job.name}")

# 3. 启动 Logistic Regression
print(f"📈 [3/3] Launching Logistic Regression...", end=" ")
lr_tuner.fit({'train': train_input}, wait=False)
print(f"✅ Job ID: {lr_tuner.latest_tuning_job.name}")

print("\n✨ 所有任务已在云端并行运行！正在启动智能监控面板...\n")
print("-" * 90)
print(f"{'Time':<10} | {'Random Forest':<20} | {'SVM':<20} | {'Logistic Regression':<20}")
print("-" * 90)

# ==========================================
# 🕵️ 实时监控循环 (包含结果处理)
# ==========================================

# 用于记录哪些任务已经处理过（打印过结果），避免重复打印
processed_jobs = {'RF': False, 'SVM': False, 'LR': False}

try:
    while True:
        # 1. 获取最新状态
        rf_st = get_tuning_job_status(rf_tuner)
        svm_st = get_tuning_job_status(svm_tuner)
        lr_st = get_tuning_job_status(lr_tuner)
        
        current_time = datetime.now().strftime('%H:%M:%S')
        print(f"{current_time:<10} | {rf_st:<20} | {svm_st:<20} | {lr_st:<20}")
        
        # 2. 检查 RF 结果
        if rf_st in ['Completed', 'Failed', 'Stopped'] and not processed_jobs['RF']:
            if rf_st == 'Completed':
                print_best_model_info(rf_tuner, "Random Forest")
            elif rf_st == 'Failed':
                print(f"\n❌ Random Forest 任务失败！")
                fetch_latest_error_log(bucket_name)
            processed_jobs['RF'] = True # 标记为已处理

        # 3. 检查 SVM 结果
        if svm_st in ['Completed', 'Failed', 'Stopped'] and not processed_jobs['SVM']:
            if svm_st == 'Completed':
                print_best_model_info(svm_tuner, "SVM")
            elif svm_st == 'Failed':
                print(f"\n❌ SVM 任务失败！")
                fetch_latest_error_log(bucket_name)
            processed_jobs['SVM'] = True

        # 4. 检查 LR 结果
        if lr_st in ['Completed', 'Failed', 'Stopped'] and not processed_jobs['LR']:
            if lr_st == 'Completed':
                print_best_model_info(lr_tuner, "Logistic Regression")
            elif lr_st == 'Failed':
                print(f"\n❌ Logistic Regression 任务失败！")
                fetch_latest_error_log(bucket_name)
            processed_jobs['LR'] = True

        # 5. 退出条件：所有任务都已处理完毕
        if all(processed_jobs.values()):
            print("-" * 90)
            print("\n🎉 所有调参任务已结束！")
            break
            
        time.sleep(30) # 每30秒刷新一次

except KeyboardInterrupt:
    print("\n⚠️ 监控已手动停止（任务仍在 AWS 后台继续运行）。")


--- 🚀 Step 5: Launching Single Debug Job ---

🌲 Launching Random Forest (Debug Mode: 1 Job)... (Timestamp: 19:08:12)


Cloning into '/var/folders/py/wsw9dv2n5g70rl8sm3pxxjg40000gn/T/tmppem1bg1l'...
Already on 'main'


Your branch is up to date with 'origin/main'.


No finished training job found associated with this estimator. Please make sure this estimator is only used for building workflow config


.......................................!
✅ Training Job Completed Successfully.


⏳ Waiting 10 seconds for S3 consistency...

🔎 [Log Fetcher] Looking for logs in s3://sleep-disorder-mlops-bucket/debug_logs/ ...


/Users/zhengchengsheng/opt/anaconda3/envs/mlops-env/lib/python3.9/site-packages/boto3/compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)


📄 [Log Fetcher] Found latest log: debug_logs/train_failure_log_20251203_111130.txt (Last Modified: 2025-12-03 11:11:31+00:00)
=== TRAINING SESSION STARTED: 2025-12-03 11:10:56.715676 ===
--- 🚀 SCRIPT START ---
INFO: Arguments: Namespace(model_type='"random_forest"', n_estimators=104, C=1.0, kernel='rbf', train='/opt/ml/input/data/train', model_dir='/opt/ml/model')
INFO: Env SM_CHANNEL_TRAINING: None
INFO: Effective Data Path: /opt/ml/input/data/train

📦 [INIT] Start environment cleanup & installation...
   --- Purging pre-installed libraries: ['numpy', 'pandas', 'scikit-learn', 'joblib']...
   ✅ Cleanup complete.
   - Installing new: numpy==1.26.4 ...
   - Installing new: pandas==2.2.0 ...
   - Installing new: scikit-learn==1.4.0 ...
   - Installing new: matplotlib ...
   - Installing new: seaborn ...
   - Installing new: joblib ...
   - Installing new: wandb ...
✅ [INIT] Fresh dependencies installed.

🔄 [IMPORT] Loading ML libraries...
✅ [IMPORT] src.data_processor loaded.

--- 1. Dat

In [8]:
import boto3
from botocore.exceptions import NoCredentialsError, ClientError
import sys

# ⚠️ 将目标桶名称固定为你自己的 S3 桶
TARGET_BUCKET_NAME = 'sleep-disorder-mlops-bucket' 

def list_target_bucket_files(bucket_name):
    """
    列出指定 S3 存储桶中的所有文件路径 (Key)。
    """
    print(f"\n=======================================================")
    print(f"| 📦 正在检查目标存储桶: {bucket_name}")
    print(f"=======================================================")
    
    try:
        s3 = boto3.client('s3')
        paginator = s3.get_paginator('list_objects_v2')
        pages = paginator.paginate(Bucket=bucket_name)
        
        found_model = False
        found_log = False
        
        for page in pages:
            contents = page.get('Contents', [])
            if contents:
                for obj in contents:
                    file_key = obj['Key']
                    print(f" - {file_key}")
                    
                    if file_key.endswith('model.tar.gz'):
                        found_model = True
                    if 'debug_logs/' in file_key:
                        found_log = True

        print(f"-------------------------------------------------------")
        print(f"✅ 模型文件 (.tar.gz) 存在状态: {'是' if found_model else '否'}")
        print(f"✅ 调试日志 (debug_logs) 存在状态: {'是' if found_log else '否'}")
            
    except ClientError as e:
        error_code = e.response['Error']['Code']
        print(f"❌ 错误：无法访问存储桶 {bucket_name}。错误代码: {error_code}。请检查权限。", file=sys.stderr)
    except NoCredentialsError:
        print("❌ 致命错误：找不到 AWS 凭证。", file=sys.stderr)
    except Exception as e:
        print(f"❌ 发生未知错误: {e}", file=sys.stderr)

if __name__ == '__main__':
    list_target_bucket_files(TARGET_BUCKET_NAME)


| 📦 正在检查目标存储桶: sleep-disorder-mlops-bucket
 - debug_logs/full_log_capture_20251203_075849.txt
 - debug_logs/sagemaker_connectivity_test_20251203072403.txt
 - debug_logs/train_failure_log_20251203_083143.txt
 - debug_logs/train_failure_log_20251203_084721.txt
 - debug_logs/train_failure_log_20251203_085633.txt
 - debug_logs/train_failure_log_20251203_090222.txt
 - debug_logs/train_failure_log_20251203_090914.txt
 - debug_logs/train_failure_log_20251203_091129.txt
 - debug_logs/train_failure_log_20251203_111130.txt
 - raw_data/sleep_data.csv
 - sagemaker-scikit-learn-2025-12-03-11-08-12-878/source/sourcedir.tar.gz
 - sagemaker-tuning-output/rf-tuning-251203-1908-001-d11209b3/output/model.tar.gz
-------------------------------------------------------
✅ 模型文件 (.tar.gz) 存在状态: 是
✅ 调试日志 (debug_logs) 存在状态: 是
